In [1]:
import pandas as pd
import numpy as np

print("="*70)
print(" SPOTIFY DATA CLEANING & PREPARATION")
print("="*70)

 SPOTIFY DATA CLEANING & PREPARATION


In [2]:
# 1. LOAD RAW DATA
print("STEP 1: Loading Raw Data...")

df = pd.read_excel('data/raw/Spotify_data.xlsx')

print(f"✓ Loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")

# Keep original for comparison
df_original = df.copy()

STEP 1: Loading Raw Data...
✓ Loaded: 520 rows × 20 columns


In [3]:
# 2. STANDARDIZE COLUMN NAMES
print("\n" + "="*70)
print("STEP 2: Standardizing Column Names")
print("="*70)

print("\nOriginal columns:")
print(list(df.columns))

# Clean column names: lowercase, remove spaces
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

print("\nCleaned columns:")
print(list(df.columns))

# 3. HANDLE MISSING VALUES
print("\n" + "="*70)
print("STEP 3: Handling Missing Values")
print("="*70)

missing = df.isnull().sum()
if missing.sum() == 0:
    print("✓ No missing values found!")
else:
    print("\nMissing values found:")
    print(missing[missing > 0])
    
    # Fill strategy for text columns
    for col in df.select_dtypes(include=['object']).columns:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna('Unknown')
            print(f"  → Filled {col} with 'Unknown'")
    
    # Fill strategy for numeric columns
    for col in df.select_dtypes(include=['number']).columns:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].median())
            print(f"  → Filled {col} with median value")



STEP 2: Standardizing Column Names

Original columns:
['Age', 'Gender', 'spotify_usage_period', 'spotify_listening_device', 'spotify_subscription_plan', 'premium_sub_willingness', 'preffered_premium_plan', 'preferred_listening_content', 'fav_music_genre', 'music_time_slot', 'music_Influencial_mood', 'music_lis_frequency', 'music_expl_method', 'music_recc_rating', 'pod_lis_frequency', 'fav_pod_genre', 'preffered_pod_format', 'pod_host_preference', 'preffered_pod_duration', 'pod_variety_satisfaction']

Cleaned columns:
['age', 'gender', 'spotify_usage_period', 'spotify_listening_device', 'spotify_subscription_plan', 'premium_sub_willingness', 'preffered_premium_plan', 'preferred_listening_content', 'fav_music_genre', 'music_time_slot', 'music_influencial_mood', 'music_lis_frequency', 'music_expl_method', 'music_recc_rating', 'pod_lis_frequency', 'fav_pod_genre', 'preffered_pod_format', 'pod_host_preference', 'preffered_pod_duration', 'pod_variety_satisfaction']

STEP 3: Handling Missing

In [4]:
# 4. REMOVE DUPLICATES
print("\n" + "="*70)
print("STEP 4: Removing Duplicates")
print("="*70)

before = len(df)
df = df.drop_duplicates()
after = len(df)
removed = before - after

if removed == 0:
    print("✓ No duplicates found!")
else:
    print(f"✓ Removed {removed} duplicate rows")
    print(f"  Dataset size: {before:,} → {after:,} rows")


STEP 4: Removing Duplicates
✓ Removed 1 duplicate rows
  Dataset size: 520 → 519 rows


In [5]:
# 5. CLEAN TEXT DATA
print("\n" + "="*70)
print("🧹 STEP 5: Cleaning Text Data")
print("="*70)

print("\nCleaning text fields...")

# Strip whitespace from all text columns
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip()

# Standardize Yes/No values
yes_no_cols = ['premium_sub_willingness', 'pod_variety_satisfaction']
for col in yes_no_cols:
    if col in df.columns:
        df[col] = df[col].str.title()  # Yes/No instead of yes/no or YES/NO
        print(f"  → Standardized {col}")

print("✓ Text cleaning complete")


🧹 STEP 5: Cleaning Text Data

Cleaning text fields...
  → Standardized premium_sub_willingness
  → Standardized pod_variety_satisfaction
✓ Text cleaning complete


In [6]:
# 6. CREATE NEW FEATURES
print("\n" + "="*70)
print("⚙️ STEP 6: Creating Derived Features")
print("="*70)

# 1. Binary: Is Premium Willing (1=Yes, 0=No)
if 'premium_sub_willingness' in df.columns:
    df['is_premium_willing'] = (df['premium_sub_willingness'].str.lower() == 'yes').astype(int)
    print("✓ Created: is_premium_willing")

# 2. Binary: Is Premium User
if 'spotify_subscription_plan' in df.columns:
    df['is_premium_user'] = (df['spotify_subscription_plan'].str.lower() == 'premium').astype(int)
    print("✓ Created: is_premium_user")

# 3. Binary: Is Podcast Listener
if 'pod_lis_frequency' in df.columns:
    df['is_podcast_listener'] = (~df['pod_lis_frequency'].str.lower().isin(['never', 'rarely'])).astype(int)
    print("✓ Created: is_podcast_listener")

# 4. Engagement Score (0-10 based on listening frequency)
if 'music_lis_frequency' in df.columns:
    engagement_map = {
        'daily': 10,
        'several times a week': 8,
        'weekly': 5,
        'occasionally': 3,
        'rarely': 1
    }
    # Apply mapping (case-insensitive)
    df['engagement_score'] = df['music_lis_frequency'].str.lower().map(engagement_map)
    df['engagement_score'] = df['engagement_score'].fillna(0)
    print("✓ Created: engagement_score")

# 5. Content Diversity (Music only, Podcast only, or Both)
if 'preferred_listening_content' in df.columns:
    df['content_diversity'] = df['preferred_listening_content'].str.title()
    print("✓ Created: content_diversity")

# 6. High Satisfaction Flag (recommendations rated 4+)
if 'music_recc_rating' in df.columns:
    df['high_satisfaction'] = (df['music_recc_rating'] >= 4).astype(int)
    print("✓ Created: high_satisfaction")

# 7. User Tenure Category
if 'spotify_usage_period' in df.columns:
    def categorize_tenure(period):
        period_lower = str(period).lower()
        if 'less' in period_lower or '<' in period_lower:
            return 'New User'
        elif '6' in period_lower and '1' in period_lower:
            return 'Growing User'
        elif '1' in period_lower and '2' in period_lower:
            return 'Established User'
        elif 'more' in period_lower or '2+' in period_lower or '>' in period_lower:
            return 'Loyal User'
        else:
            return 'Unknown'
    
    df['tenure_category'] = df['spotify_usage_period'].apply(categorize_tenure)
    print("✓ Created: tenure_category")

print(f"\n✓ Total features now: {len(df.columns)}")


⚙️ STEP 6: Creating Derived Features
✓ Created: is_premium_willing
✓ Created: is_premium_user
✓ Created: is_podcast_listener
✓ Created: engagement_score
✓ Created: content_diversity
✓ Created: high_satisfaction
✓ Created: tenure_category

✓ Total features now: 27


In [7]:
# 7. DATA TYPE OPTIMIZATION
print("\n" + "="*70)
print("STEP 7: Optimizing Data Types")
print("="*70)

print(f"Memory before: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Convert numeric ratings to integer
if 'music_recc_rating' in df.columns:
    df['music_recc_rating'] = df['music_recc_rating'].astype('int8')

print(f"Memory after:  {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


STEP 7: Optimizing Data Types
Memory before: 0.67 MB
Memory after:  0.66 MB


In [8]:
# 8. VALIDATE CLEANED DATA
print("\n" + "="*70)
print("✅ STEP 8: Data Validation")
print("="*70)

print("\nValidation Checks:")
print(f"  ✓ No missing values: {df.isnull().sum().sum() == 0}")
print(f"  ✓ No duplicates: {df.duplicated().sum() == 0}")
print(f"  ✓ Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

# Check target distribution
if 'is_premium_willing' in df.columns:
    print(f"\n  Target Variable Distribution:")
    print(f"    Willing (1): {df['is_premium_willing'].sum():,} ({df['is_premium_willing'].mean()*100:.1f}%)")
    print(f"    Not Willing (0): {(~df['is_premium_willing'].astype(bool)).sum():,} ({(1-df['is_premium_willing'].mean())*100:.1f}%)")




✅ STEP 8: Data Validation

Validation Checks:
  ✓ No missing values: True
  ✓ No duplicates: True
  ✓ Shape: 519 rows × 27 columns

  Target Variable Distribution:
    Willing (1): 186 (35.8%)
    Not Willing (0): 333 (64.2%)


In [9]:
# 9. SAVE CLEANED DATA
print("\n" + "="*70)
print("STEP 9: Saving Cleaned Data")
print("="*70)

output_path = '../data/cleaned/spotify_cleaned.csv'
df.to_csv(output_path, index=False)

print(f"✓ Cleaned data saved to: {output_path}")
print(f"  → File size: {pd.read_csv(output_path).memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# 10. CLEANING SUMMARY

print("\n" + "="*70)
print("📊 CLEANING SUMMARY")
print("="*70)

print(f"""
Original Dataset:
  • Rows: {len(df_original):,}
  • Columns: {len(df_original.columns)}

Cleaned Dataset:
  • Rows: {len(df):,} ({len(df_original) - len(df)} removed)
  • Columns: {len(df.columns)} ({len(df.columns) - len(df_original.columns)} added)
  • Missing Values: 0
  • Duplicates: 0

New Features Created: {len(df.columns) - len(df_original.columns)}
  → is_premium_willing
  → is_premium_user
  → is_podcast_listener
  → engagement_score
  → content_diversity
  → high_satisfaction
  → tenure_category

✅ Data is now ready for analysis!
""")

print("="*70)
print("Next: Run notebook 03_analysis_insights.ipynb")
print("="*70)


STEP 9: Saving Cleaned Data


OSError: Cannot save file into a non-existent directory: '..\data\cleaned'

In [10]:
# 9. SAVE CLEANED DATA
print("\n" + "="*70)
print("STEP 9: Saving Cleaned Data")
print("="*70)

output_path = 'data/cleaned/spotify_cleaned.csv'
df.to_csv(output_path, index=False)

print(f"✓ Cleaned data saved to: {output_path}")
print(f"  → File size: {pd.read_csv(output_path).memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# 10. CLEANING SUMMARY
print("\n" + "="*70)
print("CLEANING SUMMARY")
print("="*70)

print(f"""
Original Dataset:
  • Rows: {len(df_original):,}
  • Columns: {len(df_original.columns)}

Cleaned Dataset:
  • Rows: {len(df):,} ({len(df_original) - len(df)} removed)
  • Columns: {len(df.columns)} ({len(df.columns) - len(df_original.columns)} added)
  • Missing Values: 0
  • Duplicates: 0

New Features Created: {len(df.columns) - len(df_original.columns)}
  → is_premium_willing
  → is_premium_user
  → is_podcast_listener
  → engagement_score
  → content_diversity
  → high_satisfaction
  → tenure_category

✅ Data is now ready for analysis!
""")

print("="*70)
print("Next: Run notebook 03_analysis_insights.ipynb")
print("="*70)


STEP 9: Saving Cleaned Data
✓ Cleaned data saved to: data/cleaned/spotify_cleaned.csv
  → File size: 0.66 MB

CLEANING SUMMARY

Original Dataset:
  • Rows: 520
  • Columns: 20

Cleaned Dataset:
  • Rows: 519 (1 removed)
  • Columns: 27 (7 added)
  • Missing Values: 0
  • Duplicates: 0

New Features Created: 7
  → is_premium_willing
  → is_premium_user
  → is_podcast_listener
  → engagement_score
  → content_diversity
  → high_satisfaction
  → tenure_category

✅ Data is now ready for analysis!

Next: Run notebook 03_analysis_insights.ipynb
